In [1]:
from pathlib import Path
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain.chains import ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer , GenerationConfig , TextStreamer , pipeline

In [3]:
DEFAULT_TEMPLATES = """
### instruction: You're a travelling support agent that is talking to a customer . Use only the caht history and the following information 
{context}
to answer in a helpful manner to the queries . If you don't know the answer - say that you don't know.
Keep your replies short , compassionate and informative .
{chat_history}
### Input:{question}
### Response :""".strip()


class Chatbot:
    def __init__(
        self,
        text_pipeline:HuggingFacePipeline,
        embeddings:HuggingFaceEmbeddings,
        documents_sir:Path,
        prompt_template:ste=DEFAULT_TEMPLATE,
        verbrose:bool = False,):
        
        prompt = PromptTemplate(
        input_variable = ["context","question","chat_history"],
        template=prompt_template,)
        
        self.chain =self.create_chain(text_pipeline,prompt,verbrose)
        self.db = self.embed_data(documents_dir,embeddings)
        
    def _create_chain(self,
                     text_pipeline:HuggingFacePipeline,
                     prompt = PromptTemplate,
                     verbrose:bool = False,):
        memory = ConversationBufferMemory(
                 memory_key='chat_history',
                 human_prefix="### Input"
                 ai_prefix="### Response",
                 input_key ="question",
                 output_key="output_text",
                 return_message=False
                 )
        return load_qa_chain(
               text_pipeline,
               chain_type="stuff",
               prompt=prompt,
               memory=memory,
               verbrose=verbrose)
    def _embed_data(
         self,documents_dir:Path,embedings:HuggingFaceEmbeddings)->Chroma:
        loader =DirectoryLoader(documnet_dir,glob="**/*txt")
        documents =loader.load()
        text_splitter = CharacterTextSplitter(chunk_size=512,chunk_overlap=0)
        texts = texts_splitter.split_documents(documents)
        return Chroma.from_documents(texts,embeddings)
    def __call__(self,user_input:str)->str:
        docs = self.db.similarity_search(user_input)
        return self.chain.run({"input_documents":docs,"questions":user_input})
    
    
    
    

SyntaxError: invalid syntax (1020104395.py, line 34)

# Coversational Chain

In [10]:
streamer = TextStreamer(tokenizer,skip_prompt=True,skip_special_tokens=True,use_multiprocessing=False)

NameError: name 'tokenizer' is not defined

In [14]:
pipe = pipeline('text-generation',
#                 model=model,
                max_length = 2048,
                temperature=0.5,
                top_p=0.95,
#                 streamer=streamer
#                 generation_config=generation_config,
                batch_size=1,
                repetation_penalty=1.15,
               )

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


c:\users\subra\appdata\local\programs\python\python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\subra\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
template = """
### instruction: You're a travelling support agent that is talking to a customer . Use only the caht history and the following information 
{context}
to answer in a helpful manner to the queries . If you don't know the answer - say that you don't know.
Keep your replies short , compassionate and informative .
{chat_history}
### Input:{question}
## Respose""".strip()

In [7]:
prompt = PromptTemplate(
        input_variables = ["context","question","chat_history"],template=template)

In [30]:
memory = ConversationBufferMemory(
        memory_key="chat_history",
        human_prefix="##Input",
        ai_prefix = "### Response",
        output_key="answer",
        return_messages=True,

)

In [18]:
# chain = ConversationalRetrievalChain.from_llm(
#         pipe,
#         chain_type="stuff",
#         retriever=db.as_retriever(),
#         memory =memory,
#         combine_docs_chain_kwargs={'prompt':prompt},
#         return_source_documents=True,
#         verbrose=True,)

In [19]:
# from transformers import pipeline
# translator = pipeline('text-generation',model="TheBloke/Nous-Hermes-13B-GPTQ",device='cpu')

In [21]:

from transformers import AutoTokenizer , GenerationConfig , TextStreamer , pipeline , TFAutoModelForQuestionAnswering
import tensorflow as tf


In [22]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [34]:
context= "For a multi-city search on Skyscanner, simply click the 'Multi-city' button at the start of your search. From there, you can input your desired travel dates, destinations, and additional flights. Although the price alert option isn't currently available for multi-city searches, the Skyscanner team is actively working on introducing this feature in the near future. To learn more about how to navigate this new feature, refer to the provided article. Skyscanner values user feedback to enhance their services, and you can share your thoughts by clicking the 'Contact Us' button below the article."
qn =["How do I search for multiple destinations?"]

for question in qn:
    inputs = tokenizer(question,context,add_special_tokens=True,return_tensors='tf')
    input_ids=inputs["input_ids"].numpy()[0]
    
    outputs = model(inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    
    answer_start =tf.argmax(
    answer_start_scores,axis=1).numpy()[0]
    
    answer_end = (tf.argmax(answer_end_scores,axis=1)+1).numpy()[0]
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))
    print(f"question:{question}")
    print(f"Answer: {answer}")
    
    
    

question:How do I search for multiple destinations?
Answer: simply click the ' multi - city ' button at the start of your search
